In [17]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Resize
from PIL import Image
import os

In [18]:
class CustumDataset(Dataset):
    def __init__(self, root_dir, data_type= "train", transforms = None):
        super().__init__()
        self.root_dir = root_dir
        self.img_dir = os.path.join(root_dir, "train", "images")
        self.label_dir = os.path.join(root_dir, "train", "labels")
        self.images = os.listdir(self.img_dir)
        self.labels = os.listdir(self.label_dir)
        self.valid_data = self._remove_unlabeled()
        self.transforms = transforms


    def _remove_unlabeled(self):
        valid_pairs = []
        for image in self.images:
            label = image.replace(".jpg", ".txt")
            if label in self.labels:
                valid_pairs.append([image,label])
        return valid_pairs
            

    def __len__(self):
        return len(self.valid_data)

    def __getitem__(self, idx):
        
        image_path = os.path.join(self.img_dir , self.valid_data[idx][0])
        label_path = os.path.join(self.label_dir , self.valid_data[idx][1])

        image = Image.open(image_path)
        labels = []
    
        with open(label_path, "r") as file:
            for line in file:
                c_bbox = list(map(float,line.strip().split()))
                labels.append(c_bbox)
                
        if len(labels) > 5: 
            labels = labels[:5]
        elif len(labels)<5:
            n = 5-len(labels)
            labels.extend(n*[[0,0,0,0,0]])

            
        if self.transforms:
            image = self.transforms(image)
            labels = self.target_transform(image, labels)

        return image, labels

    def target_transform(self,image,labels):
        _, height, width = image.shape

        for label in labels:
            label[1] /= width
            label[2] /= height
            label[3] /= width
            label[4] /= height
        if self.transforms: 
            return torch.tensor(labels)
        return labels
        

        
        

In [19]:
transform = transforms.Compose([Resize((64,64)), ToTensor()])
train_dataset = CustumDataset("brain-tumor", transforms =transform )
tr_dataset, vl_datset = random_split(train_dataset, [0.8, 0.2])

In [20]:
trainLoader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

In [21]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2d_1 = nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 3) # input dim = 64*64
        self.conv2d_2 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3) 
        self.conv2d_3 = nn.Conv2d(in_channels = 128, out_channels = 64, kernel_size = 3) 
        self.conv2d_4 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 3)
        self.fc1 = nn.Linear(32*13*13, 16)
        self.fc2 = nn.Linear(16, 1)
        self.maxpool  = nn.MaxPool2d(2,2)
        self.activation1 = nn.ReLU()
        self.activation2 = nn.Sigmoid()

    def forward(self,x):
        x = self.activation1(self.conv2d_1(x)) #dimensions in = 64*1*64*64, out = 64*64*62*62
        x = self.activation1(self.conv2d_2(x))  # in = 64*64*62*62, out = 64*128*60*60
        x = self.maxpool(x) # in =  64*128*60*60 out = 64*128*30*30
        x = self.activation1(self.conv2d_3(x)) # in = 64*128*30*30, out = 64*64*28*28
        x = self.activation1(self.conv2d_4(x)) # in = 64*64*28*28, out = 64*32*26*26
        x = self.maxpool(x) # in = 64*32*26*26 out = 64*32*13*13
        x = x.view(x.size(0), -1)  # in = 64*32*13*13 out = 64* (32*13*13)
        x = self.fc1(x) #64*16
        x = self.activation2(self.fc2(x)) #64*1
        return x
        
        
        

In [22]:
model = Model()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [23]:
loss_fn = nn.BCELoss()

In [24]:
num_itr = 100
for epoch in range(1, 1 + num_itr):
    cum_loss = 0  # Initialize cumulative loss for the epoch
    num_batches = len(trainLoader)  # Number of batches per epoch

    for idx, batch in enumerate(trainLoader):
        images = batch[0]
        raw_labels = batch[1]
        all_labels = raw_labels[:, :, 0]
        labels = (all_labels == 1).any(dim=1)
        labels = labels.unsqueeze(1)

        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output.float(), labels.float())
        
        loss.backward()
        optimizer.step()
        
        cum_loss += loss.item()  # Add the scalar loss value (loss.item())

    avg_loss = cum_loss / num_batches  # Average loss over all batches in the epoch
    print(f"Epoch: {epoch}, Average Loss: {avg_loss}")


Epoch: 1, Average Loss: 0.6922910170895713
Epoch: 2, Average Loss: 0.6921563020774296
Epoch: 3, Average Loss: 0.692342106785093
Epoch: 4, Average Loss: 0.6927987209388188
Epoch: 5, Average Loss: 0.6921010783740452
Epoch: 6, Average Loss: 0.6921735533646175
Epoch: 7, Average Loss: 0.6921478041580745
Epoch: 8, Average Loss: 0.6919499678271157
Epoch: 9, Average Loss: 0.6918479076453617
Epoch: 10, Average Loss: 0.6914678471429008
Epoch: 11, Average Loss: 0.6915662331240517
Epoch: 12, Average Loss: 0.6915951498917171
Epoch: 13, Average Loss: 0.6908530465194157
Epoch: 14, Average Loss: 0.6906189790793827
Epoch: 15, Average Loss: 0.6904201039246151
Epoch: 16, Average Loss: 0.6896374268191201
Epoch: 17, Average Loss: 0.6893240213394165
Epoch: 18, Average Loss: 0.6879519011293139
Epoch: 19, Average Loss: 0.6866547380174909
Epoch: 20, Average Loss: 0.6841665889535632
Epoch: 21, Average Loss: 0.6818379887512752
Epoch: 22, Average Loss: 0.6782106373991285
Epoch: 23, Average Loss: 0.676244288682937